# LSE ST451: Bayesian Machine Learning
## Author: Kostas Kalogeropoulos

## Week 4: Classification

Topics covered 
 - Working with 'while' loops in Python
 - Coding Newton-Rapshon optimisation in Python
 - Finding Maximum Likelihood Estimates of logistic regression coefficients
 - Fitting Bayesian logistic Regression models and summarising the posterior of their coefficients
 - Calculate the model evidence, BIC for model choice under Bayesian logistic regression 
 - Evaluate predictive performance for binary data: missclassification rate, sensitivity, specificity, ROC curves and area under them, log scoring rule
 - Fitting linear discriminant analysis

We begin with loading the necessary libraries. We will pretty much the same libraries as next week with three additions: **scipy.optimize** for optimisation, **sklearn.discriminant_analysis** and some merics for prediction assessment from **sklearn.metrics**.

In [ ]:
import pandas as pd
import numpy as np
from scipy import optimize
import scipy as sc
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt #for plots
%matplotlib inline
from sklearn.metrics import accuracy_score, log_loss, roc_curve, roc_auc_score 
import statsmodels.api as sm

### Logistic Regression via Maximum Likelihood

Model directly the probability $p(y=1|x)$ for binary data $y \in \{0,1\}$ for some covariates $X$. There is no closed form for the MLE but we can solve approximately with an optimizer

#### Breast Cancer Survival Data

The dataset contains cases from a study that was conducted between
   1958 and 1970 at the University of Chicago's Billings Hospital on
   the survival of patients who had undergone surgery for breast
   cancer.


Four columns 

1. Age of patient at time of operation (numerical)
2. Patient's year of operation (year - 1900, numerical)
3. Number of positive axillary nodes detected (numerical)
4. Survival status (class attribute)
     1 = the patient survived 5 years or longer
     2 = the patient died within 5 year

See more info at the UCI repository [page](https://archive.ics.uci.edu/ml/machine-learning-databases/haberman/haberman.names). 

In [ ]:
url_hospital = "https://archive.ics.uci.edu/ml/machine-learning-databases/haberman/haberman.data"
data1 = pd.read_csv(url_hospital, header=None,
        names=['age', 'year', 'nodes_detected', 'survival_status'])
data1.head()

#### Train/Test split

In [ ]:
# define data and create train test splits
data1['y'] = data1.survival_status.replace(2, 0).values
n = len(data1['y'])
data1['x0'] = np.ones(n)
predictors = ['x0','age','nodes_detected']
target = ['y']
X_train, X_test, y_train, y_test = train_test_split(data1[predictors], data1[target],
                        test_size=0.33, random_state=1)
n,p = X_train.shape

#### MLE using an optimizer

The accompanying chapter in **Bishop is 4.3.2**.

In our setting we have a binary (2-class) classification problem with a sample size $n$. Our date consistts of pairs of labels and features $\{(y_1,x_1), \ldots, (y_n,x_n)\}$ where $y_i \in \{0,1\}$ and $x_i \in R^m$. We are interested in the probability $p(y=1|x)$.

The logistic regression model uses the logistic function $\sigma(\cdot) : (-\infty, -\infty) \rightarrow  (0,1) $ (also known as sigmoid)
$$\sigma(x) =\big( \frac{1}{1+e^{-x}} \big)$$


Note that the inverse of the the logistic function $\sigma(\cdot)^{-1}$, in the sense of their composition equaling the identity map, is the logit function $$\text{logit}(a) = \log\big( \frac{a}{1-a} \big)$$

Specifically, in the context of logistic regression, the function we need is the following

$$
\sigma(x, \beta) = \big( \frac{1}{1+e^{-x \beta}} \big)
$$

which we implement below.

In [ ]:
def sigma(x,beta):
    xbeta = x.dot(beta)
    d = 1. + np.exp(-xbeta)
    return d**(-1)

The Logistic Regression likelihood is given by the following formula

\begin{align}
p( y=1| \beta,x) &= \prod_{i=1}^n f(x_n,\beta)^{y_n} \cdot [1-f(x_n,\beta)]^{(1-y_n)} \\
&= \prod_{i=1}^n f_n^{y_n} \cdot (1-f_n)^{(1-y_n)} \\
\end{align}

where we define for convenience of notation $f_n := f(x_n,\beta)$ .

Hence the negative log-likelihood (NLL) becomes
\begin{align}
\log p( y=1| \beta,x) &= \sum_{i=1}^n \{ y_n \log f_n + (1-y_n) \log(1-f_n) \}
\end{align}
which we code as a function below

In [ ]:
def nll(beta, x, y):
    t = y.T.dot(np.log(sigma(x,beta)))+ (1-y).T.dot(np.log(1.-sigma(x,beta)))
    return -t

Next we will find the MLE by minimising the negagive log-likelihood using the **Newton-Raphson** algorithm. This algorithm required the gradient ($f^{\prime}$) and the Hessian matrix. Below we provide these functions.

In [ ]:
#function to find Hessian
def fprime(beta,x,y):
    return  x.T.dot(sigma(x,beta)-y)
def Hessian(beta,x):  
    S = np.diag(sigma(x,beta)*(1-sigma(x,beta)))
    hes = x.T.dot(S.dot(x))
    return hes
def NewtonRaphsonLogistReg(beta0,x,y,maxiter,tolerance):
    i = 0
    beta = beta0
    NegLogLike = nll(beta,x,y)
    AbDiff = 1; #anything bigger than tolerance would do here
    while (AbDiff> tolerance) & (i<maxiter):
        i =i+1;
        print('iteration ',i,' Negative Log likelihood ',NegLogLike, ' AbDiff ', AbDiff)
        grad = fprime(beta,x,y)
        H = Hessian(beta,x)
        H_inv = sc.linalg.inv(H) 
        beta = beta - H_inv.dot(grad)
        NegLogLike_new = nll(beta,x,y)
        AbDiff = np.abs(NegLogLike_new-NegLogLike)
        NegLogLike = NegLogLike_new
    if (iter == maxiter):
        print('Did not Converge') 
    return beta, H_inv

Now we can apply these functions to our data to obtain the MLE

In [ ]:
n,p = X_train.shape
beta0 = np.zeros(p)
tolerance = 1e-05
maxiter = 100 
beta_mle, cov = NewtonRaphsonLogistReg(beta0,X_train, y_train['y'],maxiter,tolerance)
print(beta_mle)

We will also compute 95% confidence intervals by using the Hessian which is the inverse of the MLE covariance. We will present the results via a pandas data frame

In [ ]:
#Invert for the covariance and find 95% CIs
#cov = sc.linalg.inv(H)
se = np.sqrt(np.diag(cov))
lower95 = beta_mle - 1.96*se
upper95 = beta_mle + 1.96*se

#present the output via a pandas data frame
results = np.column_stack([beta_mle,se,lower95,upper95])
col = ['coefficient','se','lower 95% bound','upper 95% bound']
ind = ['intercept','age','nodes_detected']
results = pd.DataFrame(results,columns = col,index=ind)
results

To confirm we got it right we will also check with the function Logit from the statsmodels library

In [ ]:
predictors = ['age','nodes_detected']
Xt = X_train[predictors]
Xt = sm.add_constant(Xt)
yt = y_train['y']
model = sm.Logit(yt, Xt).fit()
model.summary()

The function LogisticRegression from sklearn can also be used. It is more robust and efficient for large datasets

In [ ]:
predictors = ['age','nodes_detected']
Xt = X_train[predictors]
logreg = LogisticRegression(fit_intercept=True,solver='lbfgs')
logreg.fit(Xt,yt)
intercept = np.array([logreg.intercept_])[0,]
coef = np.array(logreg.coef_)[0,].T
beta_mle = np.hstack((intercept,coef))
print(beta_mle)

### Laplace Approximation (Bishop 4.5)

Bayesian Logistic Regression is not availabe in a closed form. One method is to use a Laplace approximation according to which we approximate the posterior $p(\beta | x, y)$ with a Gaussian. The Gaussian distribution is characterised by two parameters, the mean and the covariance. The mean $\beta_{\text{MAP}}$ is given by the MAP of the unormalized likelihood times prior, while the covariance is the Hessian evaluated $\beta_{\text{MAP}}$. 

The posterior is given by equation 4.141 and 4.142 which we implement below

Assume a Gaussian prior and get the following approximate posterior 

\begin{align}
\beta & \sim N(m_0, S_0^{-1}) \; \text{(prior)}\\
p(\beta|x,y) & \sim  N(\hat{\beta}, S^{-1})  \; \text{(posterior)}
\end{align}

where
\begin{align}
\beta_{\text{MAP}} & = \text{argmin}_{\beta} \log p(\beta|y,x), \; \log p(\beta|y,x) = \log p(y|\beta,x) + \log p(\beta) \\
S & = -\nabla^2 \log p(\beta|y,x) |_{\beta_{\text{MAP}}}
\end{align}

In [ ]:
def neg_post(beta, x, y, m0, S0):
    logprior =  - .5* (beta-m0).T @ S0 @ (beta-m0)
    return  nll(beta, x, y) -  logprior
def neg_post_prime(beta, x, y, m0, S0):
    return  x.T.dot(sigma(x,beta)-y)+S0.dot(beta-m0)
def laplace_prec(beta, x, S0):
    n = x.shape[0]  
    S = np.diag(sigma(x,beta)*(1-sigma(x,beta)))
    prec = S0.values + x.T.dot(S.dot(x))    
    return prec
def NewtonRaphsonBayesLogReg(beta0,x,y,m0,S0,maxiter,tolerance):
    i = 0
    beta = beta0
    NegLogPost = neg_post(beta, x, y, m0, S0)
    AbDiff = 1; #anything bigger than tolerance would do here
    while (AbDiff> tolerance) & (i<maxiter):
        i =i+1;
        print('iteration ',i,' Negative Log Posterior ',NegLogPost, ' AbDiff ', AbDiff)
        grad = neg_post_prime(beta, x, y, m0, S0)
        H = laplace_prec(beta, x, S0)
        H_inv = sc.linalg.inv(H) 
        beta = beta - H_inv.dot(grad)
        NegLogPost_new = neg_post(beta, x, y, m0, S0)
        AbDiff = np.abs(NegLogPost_new-NegLogPost)
        NegLogPost = NegLogPost_new
    if (iter == maxiter):
        print('Did not Converge') 
    return beta, H_inv

In [ ]:
n,p = X_train.shape
m0 = np.zeros(p)
S0 = X_train.T.dot(X_train)/n
beta0 = np.zeros(p)
tolerance = 1e-05
maxiter = 100 
beta_map, cov = NewtonRaphsonBayesLogReg(beta0,X_train, y_train['y'],
                                                  m0,S0,maxiter,tolerance)
print(beta_map)
cov

In [ ]:
#95% credible intervals
se = np.sqrt(np.diag(cov))
lower95 = beta_map - 1.96*se
upper95 = beta_map + 1.96*se

#present the output via a pandas data frame
results = np.column_stack([beta_map,se,lower95,upper95])
col = ['post mean','post se','lower 95% bound','upper 95% bound']
ind = ['intercept','age','nodes_detected']
results = pd.DataFrame(results,columns = col,index=ind)
results

### Activity 1

Consider the model that also has the variable year as a covariate. Provide the ouput from the MLE and Bayesian (via Laplace approximation) approaches. 

### Answer

Put your code below

### Evaluate predictive performance for Logistic regression

In [ ]:
#Calculate model evidence and BIC
n,p = X_train.shape
logEv = -neg_post(beta_map, X_train, y_train, m0, S0) + (p/2)*np.log(2*np.pi) + (1/2)*np.log(sc.linalg.det(cov))
BIC = -nll(beta_map, X_train, y_train) -(p/2)*np.log(n)
logEv, BIC

In [ ]:
#prediction assessment for MLE approach
predict_probs = sigma(X_test,beta_mle)
predict_class = (predict_probs > .5)

AccRate_MLE = np.sum(predict_class==y_test['y'])/len(y_test) 
LS_MLE = -(y_test.T.dot(np.log(predict_probs))+(1-y_test).T.dot(np.log(1-predict_probs)))

print(AccRate_MLE, LS_MLE)
print(accuracy_score(y_test,predict_class), log_loss(y_test, predict_probs, normalize=False))

fpr,tpr,thresholds = roc_curve(y_test,predict_probs)
xgrid = np.linspace(0,1,100) 
plt.plot(fpr,tpr)
plt.plot(xgrid,xgrid,'red')
AUC_MLE = roc_auc_score(y_test,predict_probs)
print(AUC_MLE)

In [ ]:
#Calculate the prediction probabilities with Bayesian Logistic regression

N = 10000 #number of Monte Carlo samples
# draw N samples from the approximate posterior (Laplace) of beta
betas = np.random.multivariate_normal(beta_map,cov,N)
Xt = X_test.values[0,]
print(np.mean(Xt.dot(betas.T)))
Xbetas = X_test.dot(betas.T)
test = np.mean(Xbetas, axis=1)
test.head()

#apply sigmoid
d = 1/(1. + np.exp(-Xbetas))
pred_probs_blr = np.mean(d, axis=1)
plt.plot(pred_probs_blr,'.')
plt.plot(predict_probs,'.')

In [ ]:
#Evaluate predictive performance of Bayesian Logistic Regression

pred_class_blr = (pred_probs_blr > .5)

AccRate = np.sum(pred_class_blr==y_test['y'])/len(y_test) 
LS = -(y_test.T.dot(np.log(pred_probs_blr))+(1-y_test).T.dot(np.log(1-pred_probs_blr)))

fpr,tpr,thresholds = roc_curve(y_test,pred_probs_blr)
xgrid = np.linspace(0,1,100) 
plt.plot(fpr,tpr)
plt.plot(xgrid,xgrid,'red')
AUC = roc_auc_score(y_test,pred_probs_blr)
print(AccRate, LS, AUC)
print(AccRate_MLE, LS_MLE, AUC_MLE)

### Activity 2

Consider 2 logistic regression models: 

 - M1: with covariates *age* and *nodes detected*.
 - M2: with covariates *year*, *age* and *nodes detected*.
 
 Compare them using the log-evidence, the BIC, the area under the ROC curve and the log scoring rule.

### Answer

Put your code below

### Linear Discriminant Analysis

Compute the classification probability $p(y=1|x)$ by modeling the joint probability $p(y,x)$. That's called a generative model. 

#### Train/Test split

In [ ]:
# define data and create train test splits
X = data1.loc[:,['age', 'nodes_detected']].values
y = data1.survival_status.replace(2, 0).values.astype(int)
X_train, X_test, y_train, y_test = train_test_split(X, y,
                        test_size=0.33, random_state=1)

In [ ]:
X_train.shape

We will first present a solution using `sklearn`. The following simple commands suffice to train a Linear Discriminant Analysis classifier using the `sklearn` (also known as sci-kit package).

In our setting we have a binary (2-class) classification problem with a sample size $n$. Our date consistts of pairs of labels and features $\{(y_1,x_1), \ldots, (y_n,x_n)\}$ where $y_i \in \{0,1\}$ and $x_i \in R^m$. We are interested in the probability $p(y=1|x)$.

The formulas used are the following equations in Bishop

* Prior probability of class 0 : 4.73
* Mean of features 4.75, 4.76
* Covariance matrix 4.78

which are implemented by the sklearn package.

In [ ]:
clf = LinearDiscriminantAnalysis(solver='lsqr',store_covariance=True)
clf.fit(X_train, y_train)
clf.means_, clf.priors_, clf.covariance_

We can also predict the class of new individuals based on their $X$'s

In [ ]:
pred_probs_lda = clf.predict_proba(X_test)
pred_probs_lda.shape

Essentially we only need the probability of y=1 (2nd column). Probability of y=0 is just one minus that.

In [ ]:
pred_probs_lda = pred_probs_lda[:,1]

### Activity 3

Fit logistic regression and LDA models on the training data and evaluate their predictive performance (ROC, area under the ROC, log score) on the test data. Use *year*, *age* and *nodes_detected* as covariates.

### Answer

Put your code below